# DFNN-based_predictor

In this demo, we will construct a Deep feedforward neural network(DFNN)-based predictor for predicting the acticity of CES2 inhibitor. Herein, DFNN model with were carried out by pytorch.

## Import modules

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [5]:
import os
import pandas as pd
import sys
import numpy as np
os.chdir('./')
#print(os.getcwd())

## Molecular characterization

In [6]:
sys.path.append('./release/') 
from Data_preprocess import load_data,calcMCFP

Load data

In [7]:
path = "Dataset_CES2_inhibitors_pIC50.csv"

dataset,canonical_smi,canonical_mols = load_data(path)

Calculate MCFP descriptors

In [8]:
pred_data = calcMCFP(mols = canonical_mols, dataset = dataset)

## Predictor construction

In [9]:
sys.path.append('./release/') 
from DFNN_predictor import RegressionDataset,data_process,Net,fit_NN_model,NN_predict

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [11]:
# 查看是否可用GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


The train_test_split from sklearn was employed for training ML model.

In [12]:
x_train,x_test,y_train,y_test,train_dataset,test_dataset = data_process(data=pred_data, 
                                                                        test_size=0.3, random_state=43)

Define hyperparameters and training NN model

In [13]:
EPOCHS = 150
BATCH_SIZE = 64
LEARNING_RATE = 0.001
NUM_FEATURES = x_train.shape[1] #查看行数 len(X_train) 或 X_train.shape[0]；查看列数 X_train.shape[1]

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1)

# training model
loss_status,model=fit_NN_model(train_loader,test_loader,EPOCHS,BATCH_SIZE,LEARNING_RATE,NUM_FEATURES)

Begin training.


  0%|          | 0/150 [00:00<?, ?it/s]

Epoch 001: | Train Loss: 37.76263 | Test Loss: 37.33814
Epoch 002: | Train Loss: 37.37641 | Test Loss: 36.64640
Epoch 003: | Train Loss: 36.21207 | Test Loss: 35.54752
Epoch 004: | Train Loss: 36.03541 | Test Loss: 33.60041
Epoch 005: | Train Loss: 32.10538 | Test Loss: 30.04410
Epoch 006: | Train Loss: 27.86246 | Test Loss: 23.71211
Epoch 007: | Train Loss: 20.65418 | Test Loss: 14.10335
Epoch 008: | Train Loss: 9.97358 | Test Loss: 5.08433
Epoch 009: | Train Loss: 4.17402 | Test Loss: 5.04789
Epoch 010: | Train Loss: 4.72249 | Test Loss: 4.62037
Epoch 011: | Train Loss: 4.57718 | Test Loss: 3.46403
Epoch 012: | Train Loss: 3.22293 | Test Loss: 3.28951
Epoch 013: | Train Loss: 3.00133 | Test Loss: 3.21170
Epoch 014: | Train Loss: 2.67975 | Test Loss: 2.98486
Epoch 015: | Train Loss: 2.65039 | Test Loss: 2.82591
Epoch 016: | Train Loss: 2.41614 | Test Loss: 2.68407
Epoch 017: | Train Loss: 2.42474 | Test Loss: 2.51492
Epoch 018: | Train Loss: 2.59470 | Test Loss: 2.38005
Epoch 019: | T

In [17]:
y_pred_train = model(torch.from_numpy(x_train).float()).tolist()
y_pred_test = model(torch.from_numpy(x_test).float()).tolist()

print("R2 for training set:",r2_score(y_train, y_pred_train))
print("R2 for training set:",r2_score(y_test, y_pred_test))


R2 for training set: 0.7666204904101118


Save model

In [ ]:
path = "./model/DFNN.pka"

#torch.save(model.state_dict(), path)

## Predicting data

In [ ]:
sys.path.append('./release/') 
from DFNN_predictor import NN_predict

Input data

In [ ]:
path = "Demo_pre_act.csv"

dataset,canonical_smi,canonical_mols = load_data(path)
vali_data = calcMCFP(mols = canonical_mols, dataset = dataset)

Loading predictor

In [ ]:
data = vali_data  # validation set
path = "./model/DFNN.pka"

pre_act = NN_predict(data = vali_data, path =path, num_features = 128)